In [1]:
!pip install BitsandBytes

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [3]:
quantisation_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

In [4]:
from huggingface_hub import login
login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model_storage_path = '/content/drive/MyDrive/models'


Mounted at /content/drive


In [ ]:
!pip install "transformers>=4.45.1"


In [ ]:
tokenizer=AutoTokenizer.from_pretrained("klyang/MentaLLaMA-chat-7B")
mdel=AutoModelForCausalLM.from_pretrained("klyang/MentaLLaMA-chat-7B",quantization_config=quantisation_config,device_map="auto",offload_folder="offload")


CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
input="Hossam gets a cold easily. Q: Is his immune system too weak? A:"
input_ids=tokenizer(input,return_tensors="pt").input_ids.to("cuda")
outputs = mdel.generate(input_ids, max_length=128,num_return_sequences=1)
final=tokenizer.decode(outputs[0],skip_special_tokens=True)
print(final.split("A:")[-1].strip())

No, his immune system is not too weak. Reasoning: Hossam getting a cold easily does not necessarily indicate that his immune system is weak. There are many factors that can contribute to someone getting sick more easily, such as exposure to viruses, lack of sleep, stress, and poor nutrition. Without further information, it is difficult to determine if Hossam's immune system is weak or if there are other underlying factors contributing to his susceptibility to colds.


In [5]:
!pip install datasets

In [6]:
!pip install peft

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model,prepare_model_for_kbit_training
import torch

In [8]:
tokenizer=AutoTokenizer.from_pretrained("klyang/MentaLLaMA-chat-7B")
mdel=AutoModelForCausalLM.from_pretrained("klyang/MentaLLaMA-chat-7B",quantization_config=quantisation_config,device_map="auto",offload_folder="offload")
mdel=prepare_model_for_kbit_training(mdel)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
lora_config = LoraConfig(
    r=8,  # Low-rank dimension
    lora_alpha=16,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Fine-tune attention projections
    lora_dropout=0.1,  # Dropout for LoRA layers
    bias="none",  # No bias fine-tuning
    task_type="CAUSAL_LM"  # Task type (causal language modeling)
)

In [10]:
mdel = get_peft_model(mdel, lora_config)


In [13]:
dataset = load_dataset("csv", data_files={"train": "reddit_train.csv", "validation": "reddit_valid.csv"})


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [14]:
print(dataset["train"][0])

{'post': 'Post: How to avoid a relapse? I\'ve been having a particularly rough year; I attempted suicide, the love of my life left me, I failed my year at university, I\'ve been physically assaulted, I\'ve been sexually assaulted and today my grandfather died. I feel like I\'m running on empty and doing the bare minimum to survive for myself while trying to be a rock to others. My emotional resilience has been wiped out. I feel "okay" but I\'ve felt like this in the past and it has turned out that I\'ve just been lying to myself and making things worse. Any advice on how to manage/process my emotions? Or just how to better understand myself?', 'question': ' Question: The answer to the question "Does the poster suffers from depression?" is ', 'response': "yes. Reasoning: The post displays a significant amount of emotional distress, including experiencing a suicide attempt, multiple traumatic events, and the recent death of a family member. The poster describes feeling emotionally deplet

In [15]:
def tokenize_function(example):
    encodings = tokenizer(example["post"], truncation=True, padding="max_length", max_length=512)
    encodings["labels"] = encodings["input_ids"]  # Set labels to input_ids for language modeling
    return encodings


In [16]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1003 [00:00<?, ? examples/s]

Map:   0%|          | 0/430 [00:00<?, ? examples/s]

In [17]:
from transformers import Trainer, TrainingArguments


In [18]:
training_args = TrainingArguments(
    output_dir="./llama-fine-tuned",  # Output directory for model checkpoints
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=16,
    learning_rate=5e-5,  # Set an appropriate initial learning rate
    warmup_steps=500,  # Linear warmup for the first 500 steps
    weight_decay=0.01,
    num_train_epochs=3,
    fp16=True,  # Use mixed precision for faster training
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=200,
    save_total_limit=3,
    report_to="none",  # Disable reporting (e.g., WandB or TensorBoard)
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
trainer = Trainer(
    model=mdel,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

<ipython-input-19-584273b97caa>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


TrainOutput(global_step=21, training_loss=9.540472212291899, metrics={'train_runtime': 4332.2383, 'train_samples_per_second': 0.695, 'train_steps_per_second': 0.005, 'total_flos': 5.459501238976512e+16, 'train_loss': 9.540472212291899, 'epoch': 2.7142857142857144})

In [26]:
input="How to avoid a relapse? I've been having a particularly rough year; I attempted suicide, the love of my life left me, I failed my year at university, I've been physically assaulted, I've been sexually assaulted and today my grandfather died. I feel like I'm running on empty and doing the bare minimum to survive for myself while trying to be a rock to others. My emotional resilience has been wiped out. I feel "'okay'" but I've felt like this in the past and it has turned out that I've just been lying to myself and making things worse. Any advice on how to manage/process my emotions? Or just how to better understand myself?"
input_ids=tokenizer(input,return_tensors="pt").input_ids.to("cuda")
outputs = mdel.generate(input_ids,num_return_sequences=1,max_length=2048)
final=tokenizer.decode(outputs[0],skip_special_tokens=True)
print(final.strip())

How to avoid a relapse? I've been having a particularly rough year; I attempted suicide, the love of my life left me, I failed my year at university, I've been physically assaulted, I've been sexually assaulted and today my grandfather died. I feel like I'm running on empty and doing the bare minimum to survive for myself while trying to be a rock to others. My emotional resilience has been wiped out. I feel okay but I've felt like this in the past and it has turned out that I've just been lying to myself and making things worse. Any advice on how to manage/process my emotions? Or just how to better understand myself?

I'm sorry to hear about the tough year you've been through. It's understandable that you're feeling overwhelmed and struggling to cope with all the negative experiences. Here are some suggestions on how to manage and process your emotions, as well as better understanding yourself:

1. Practice self-care: Take care of your physical and emotional needs. Make sure to get en

In [21]:
trainer.save_model("./llama-fine-tuned")


In [22]:
mdel.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Lin

In [27]:
mdel.save_pretrained("D:/")
tokenizer.save_pretrained("D:/")

('D:/tokenizer_config.json',
 'D:/special_tokens_map.json',
 'D:/tokenizer.model',
 'D:/added_tokens.json',
 'D:/tokenizer.json')

In [28]:
mdel.push_to_hub("Seravian_Tuned_model")
tokenizer.push_to_hub("Seravian_Tuned_model")

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MeloRFTW/Seravian_Tuned_model/commit/8cba023561de7a4cefa7475e06c900c37d561550', commit_message='Upload tokenizer', commit_description='', oid='8cba023561de7a4cefa7475e06c900c37d561550', pr_url=None, repo_url=RepoUrl('https://huggingface.co/MeloRFTW/Seravian_Tuned_model', endpoint='https://huggingface.co', repo_type='model', repo_id='MeloRFTW/Seravian_Tuned_model'), pr_revision=None, pr_num=None)

In [29]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [32]:
!cp -r /content/llama-fine-tuned /content/drive/My\ Drive/

In [33]:
!cp -r /content/llama-fine-tuned /content/drive/My\ Drive/

In [34]:
import shutil

# Compress the folder into a zip file
shutil.make_archive("/content/llama-fine-tuned", "zip", "/content/llama-fine-tuned")


'/content/llama-fine-tuned.zip'

In [35]:
from google.colab import files

# Download the zip file
files.download("/content/llama-fine-tuned.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>